# Importing Packages

In [1]:
import numpy as np
from scipy import linalg as splinalg
import matplotlib.pyplot as plt
from scipy import sparse as sp
import scipy.sparse.linalg as splin
from functools import reduce
import itertools
from scipy import linalg
from scipy.linalg import expm, logm
from scipy.special import comb
from itertools import combinations_with_replacement, product
from collections import Counter
import copy
from scipy.linalg import ishermitian
from scipy.linalg import eigh
import scipy

# Definitions

In [66]:
Z = sp.csc_matrix([[1, 0], [0, -1]])
X = sp.csc_matrix([[0, 1], [1, 0]])
Y = sp.csc_matrix([[0, -1j], [1j, 0]])
I = sp.csc_matrix([[1, 0], [0, 1]])

params={}
params['sites'] = 4
params['I'] = sp.eye(2)
params['X'] = X
params['Y'] = Y
params['Z'] = Z

In [67]:
print(params['Y'].toarray())

[[0.+0.j 0.-1.j]
 [0.+1.j 0.+0.j]]


In [68]:
print(Y.toarray())

[[0.+0.j 0.-1.j]
 [0.+1.j 0.+0.j]]


In [69]:
H = sp.csc_matrix((2**params['sites'], 2**params['sites']))
# H.toarray()

# Heisenberg Hamiltonian 1D

In [70]:
H = sp.csc_matrix((2**params['sites'], 2**params['sites']))
for i in range(params['sites']-1):
    H += reduce(sp.kron, (sp.eye(2**i), params['X'], params['X'], sp.eye(2**(params['sites']-2-i)))) + reduce(sp.kron, (sp.eye(2**i), params['Y'], params['Y'], sp.eye(2**(params['sites']-2-i)))) + reduce(sp.kron, (sp.eye(2**i), params['Z'], params['Z'], sp.eye(2**(params['sites']-2-i))))

H

<Compressed Sparse Column sparse matrix of dtype 'complex128'
	with 40 stored elements and shape (16, 16)>

In [76]:
# eigenvalues, eigenvectors = splin.eigsh(H, k=1, which='SA')
eigenvalues, eigenvectors = np.linalg.eigh(H.toarray())

In [77]:
eigenvalues[0]

np.float64(-6.464101615137755)

In [88]:
eigenvectors
print(eigenvectors[:,0])
ground_state = eigenvectors[:, 0]  

[-0.00000000e+00-0.j -0.00000000e+00-0.j -0.00000000e+00-0.j
  1.49429245e-01+0.j  0.00000000e+00+0.j -5.57677536e-01+0.j
  4.08248290e-01+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
  4.08248290e-01+0.j -5.57677536e-01+0.j  1.88296102e-17+0.j
  1.49429245e-01+0.j -7.64157212e-18+0.j  2.04755308e-18+0.j
  0.00000000e+00+0.j]


# Covariance matrix method

In [89]:
def generate_two_site_pauli_operators(N):
    two_site_ops = []
    for i in range(N - 1): 
        X_iX_i1 = reduce(sp.kron, (sp.eye(2**i), params['X'], params['X'], sp.eye(2**(N - i - 2))))
        Y_iY_i1 = reduce(sp.kron, (sp.eye(2**i), params['Y'], params['Y'], sp.eye(2**(N - i - 2))))
        Z_iZ_i1 = reduce(sp.kron, (sp.eye(2**i), params['Z'], params['Z'], sp.eye(2**(N - i - 2))))
        two_site_ops.extend([X_iX_i1, Y_iY_i1, Z_iZ_i1])
    return two_site_ops

two_site_operators = generate_two_site_pauli_operators(params['sites'])
# print(two_site_operators)

num_ops = len(two_site_operators)
# covariance_matrix = sp.csc_matrix((num_ops, num_ops), dtype=complex)
covariance_matrix = np.zeros((num_ops, num_ops), dtype=complex)

for i in range(num_ops):
    for j in range(num_ops):
        V_i = two_site_operators[i].toarray()  # Convert sparse matrix to dense array
        V_j = two_site_operators[j].toarray()  # Convert sparse matrix to dense array
        term1 = np.vdot(ground_state, np.dot(V_i, np.dot(V_j, ground_state)))
        term2 = np.vdot(ground_state, np.dot(V_i, ground_state)) * np.vdot(ground_state, np.dot(V_j, ground_state))
        # print(term1-term2)
        covariance_matrix[i, j] = term1 - term2  # Assign scalar value

# eigenvalues, eigenvectors = splin.eigsh(covariance_matrix)
eigenvalues1, eigenvectors1 = np.linalg.eigh(covariance_matrix)

## Eigenvectors with zero eigenvalue (local operators that commute with the ground state)
zero_eigenvectors1 = eigenvectors1[:, np.isclose(eigenvalues1, 0, atol=1e-8)]

print("Eigenvectors with zero eigenvalue of the covariance matrix (two-site Pauli operators):")
for i in range(zero_eigenvectors1.shape[1]):
    print(f"Eigenvector {i + 1}:")
    print(zero_eigenvectors1[:, i])

# # Reconstruct the effective Hamiltonian in the subspace of zero eigenvectors
# # Use the zero eigenvectors to construct an effective Hamiltonian
effective_hamiltonian_covar = np.zeros((2**params['sites'], 2**params['sites']), dtype=complex)
for i in range(zero_eigenvectors1.shape[1]):
    coeff = zero_eigenvectors1[:, i]
    V_eff = sum(coeff[j] * two_site_operators[j].toarray() for j in range(num_ops))
    effective_hamiltonian_covar += V_eff

print("\nEffective Hamiltonian in the subspace of zero eigenvectors:")
print(effective_hamiltonian_covar)

Eigenvectors with zero eigenvalue of the covariance matrix (two-site Pauli operators):
Eigenvector 1:
[-0.26307262+0.j  0.89490499+0.j  0.01981398+0.j  0.26832039+0.j
 -0.11171248+0.j  0.19241526+0.j  0.08053948+0.j -0.03916902+0.j
  0.00502954+0.j]
Eigenvector 2:
[ 0.66217223+0.j  0.20258596+0.j -0.11075431+0.j -0.236642  +0.j
 -0.01149414+0.j  0.27582426+0.j  0.0280196 +0.j -0.12750953+0.j
 -0.59913771+0.j]
Eigenvector 3:
[-0.43048915+0.j -0.17183018+0.j -0.16724038+0.j -0.11891153+0.j
 -0.15033305+0.j  0.15799756+0.j  0.57988597+0.j  0.40707219+0.j
 -0.43989251+0.j]
Eigenvector 4:
[-0.10985602+0.j -0.01453274+0.j  0.23929191+0.j -0.16903119+0.j
  0.26229053+0.j -0.09982542+0.j  0.56130082+0.j -0.71241533+0.j
  0.02307921+0.j]
Eigenvector 5:
[-0.19113123+0.j -0.04759636+0.j  0.76530739+0.j  0.10558352+0.j
 -0.05942822+0.j -0.16392369+0.j -0.28309032+0.j  0.02419528+0.j
 -0.50322154+0.j]
Eigenvector 6:
[-0.36387243-0.j  0.01022673+0.j -0.4363695 +0.j -0.26483903+0.j
 -0.3800717 +0.j -

# Correlation matrix method

In [90]:
two_site_operators = generate_two_site_pauli_operators(params['sites'])

num_ops = len(two_site_operators)
# correlation_matrix = sp.csc_matrix((num_ops, num_ops), dtype = complex)
correlation_matrix = np.zeros((num_ops, num_ops), dtype = complex)

for i in range(num_ops):
    for j in range(num_ops):
        Vi = two_site_operators[i].toarray()
        Vj = two_site_operators[j].toarray()
        term11 = np.vdot(ground_state, np.dot(Vi, np.dot(Vj, ground_state)))
        term22 = 2 * np.vdot(ground_state, np.dot(Vi, ground_state)) * np.vdot(ground_state, np.dot(Vj, ground_state))
        print(term11-term22)
        correlation_matrix[i, j] = term11 - term22

# print(covariance_matrix)
# eigenvalues2, eigenvectors2 = splin.eigsh(correlation_matrix)
eigenvalues2, eigenvectors2 = np.linalg.eigh(correlation_matrix)

## Eigenvectors with zero eigenvalue (local operators that commute with the ground state)
zero_eigenvectors2 = eigenvectors2[:, np.isclose(eigenvalues2, 0, atol=1e-8)]

print("Eigenvectors with zero eigenvalue of the covariance matrix (two-site Pauli operators):")
for i in range(zero_eigenvectors2.shape[1]):
    print(f"Eigenvector {i + 1}:")
    print(zero_eigenvectors2[:, i])

# # Reconstruct the effective Hamiltonian in the subspace of zero eigenvectors
# # Use the zero eigenvectors to construct an effective Hamiltonian
effective_hamiltonian_correl = np.zeros((2**params['sites'], 2**params['sites']), dtype=complex)
for i in range(zero_eigenvectors2.shape[1]):
    coeff = zero_eigenvectors2[:, i]
    V_eff = sum(coeff[j] * two_site_operators[j].toarray() for j in range(num_ops))
    effective_hamiltonian_correl += V_eff

print("\nEffective Hamiltonian in the subspace of zero eigenvectors:")
print(effective_hamiltonian_correl)

(-0.6586892478083896+0j)
(-0.7480056452854305+0j)
(-0.7480056452854305+0j)
(-0.3631054658256804+0j)
(-0.6071224016819727+0j)
(-0.6071224016819731+0j)
(-0.6586892478083893+0j)
(-0.7480056452854302+0j)
(-0.7480056452854305+0j)
(-0.7480056452854305+0j)
(-0.6586892478083896+0j)
(-0.7480056452854305+0j)
(-0.6071224016819727+0j)
(-0.3631054658256804+0j)
(-0.6071224016819731+0j)
(-0.7480056452854302+0j)
(-0.6586892478083893+0j)
(-0.7480056452854305+0j)
(-0.7480056452854305+0j)
(-0.7480056452854305+0j)
(-0.6586892478083896+0j)
(-0.6071224016819727+0j)
(-0.6071224016819727+0j)
(-0.36310546582568043+0j)
(-0.7480056452854303+0j)
(-0.7480056452854303+0j)
(-0.6586892478083896+0j)
(-0.3631054658256804+0j)
(-0.6071224016819727+0j)
(-0.6071224016819727+0j)
(0.7777777777777777+0j)
(0.11111111111111127+0j)
(0.111111111111111+0j)
(-0.36310546582568026+0j)
(-0.6071224016819726+0j)
(-0.6071224016819727+0j)
(-0.6071224016819727+0j)
(-0.3631054658256804+0j)
(-0.6071224016819727+0j)
(0.11111111111111127+0j)
(

# Comparison

In [91]:
# np.allclose(effective_hamiltonian_correl.toarray(), effective_hamiltonian_covar.toarray(), atol=1e-8)
np.allclose(effective_hamiltonian_correl, effective_hamiltonian_covar, atol=1e-8)

False